In [ ]:
#| default_exp notifyr

In [ ]:
from nbdev.showdoc import *

In [ ]:
#| hide

from nostr_relay import web

In [ ]:
#| hide

web.run_with_uvicorn(conf_file='../nostr-relay/nostr-relay-config.yml', in_thread=True)

# Nostr Notifyr

> Using the basic client to make a simple notification bot to alert when a python function is done running or if it fails

## making a decorator that can send messages
We are going to make a decorator that will store a private key in the keychain and send an encrypted message to itself. The private key will also be assigned to the decorated function as an attribute. The user can then log into any type of nostr client that can receive encrypted DMs to get notifications about python processing results or if the function errors out. Other projects like [knockknock](https://github.com/huggingface/knockknock) offer this sort of service, but take more leg work to get an account set up. Being able to randomly generate a Nostr private key makes this quick and painless.

> **Note**: There is some concern about the safety of the current encrypted message implementation. Please do not use this module for any sort of secure communication. You can see a Github issue on the topic [here](https://github.com/nostr-protocol/nips/issues/107)

> **Another note about keyring:** This project is tested and runs on MacOS - the majority of it seems to build on linux, but seems to have some issues running on Linux due to `keyring` dependencies. The [keyring documenation](https://pypi.org/project/keyring/) may help if you attempt to debug. I believe installing [kwallet](https://en.wikipedia.org/wiki/KWallet) might be the eventual solution.

In [ ]:
#| export

from nostrfastr.client import Client
import time

First let's make a helper function that will send the message

In [ ]:
#| export

def send_nostr_message(notifyr_client: Client, message: str, recipient_pubkey_hex: str) -> None:
    """a simple function that takes a client and a message and
    sends the message to the client pubkey from the client pubkey
 
    Parameters
    ----------
    notifyr_client : Client
       A client class that will send an encrypted message for us
    message : str
       A message that will be encrypted and sent
    """
    with notifyr_client:
        event = notifyr_client.event_encrypted_message(recipient_hex=recipient_pubkey_hex,
                                                       message=message)
        notifyr_client.publish_event(event)
    pass


Next we are going to make helper functions to get and set credentials from `keyring`

In [ ]:
#| export

import keyring
from keyring.errors import NoKeyringError
from nostrfastr.nostr import PrivateKey, PublicKey

In [ ]:
show_doc(keyring.set_password)

---

### set_password

>      set_password (service_name:str, username:str, password:str)

Set password for the user in the specified service.

In [ ]:
#| export

def set_private_key(notifyr_privkey_hex: str) -> None:
   """Set the private key in the computer keyring

   Parameters
   ----------
   notifyr_privkey_hex : str
       nostr hex private key
   """
   return keyring.set_password(service_name='nostr',
                                username='notifyr',
                                password=notifyr_privkey_hex)

def get_private_key() -> str:
    """get the nostr hex private key from the computer key ring

    Returns
    -------
    str
        nostr hex private key
    """
    return keyring.get_password(service_name='nostr',
                                username='notifyr')

def delete_private_key() -> None:
    """delete the nostr hex private key from the computer key ring.
    This is not used in the decorator function, but may be used
    if need for testing
    """
    return keyring.delete_password(service_name='nostr',
                                   username='notifyr')


Let's test setting and restoring the private key

In [ ]:
#| hide
current_machine_privkey = get_private_key()

Clear the private key from keychain

In [ ]:
priv_key_hex = get_private_key()
if priv_key_hex is not None:
    delete_private_key()
assert get_private_key() is None


Try setting a new one

In [ ]:
priv_key_hex = PrivateKey().hex()
set_private_key(notifyr_privkey_hex = priv_key_hex)
assert get_private_key() == priv_key_hex

In [ ]:
#| hide
if current_machine_privkey is not None:
    set_private_key(notifyr_privkey_hex=current_machine_privkey)

Finally we write the decorator function complete with
 - nostr client handling
 - start message, success message, error message handling
 - and setting the private key to the decorator function for easy user access

In [ ]:
#| export

def convert_to_hex(pubkey: str) -> str:
    """make sure the pubkey is hex

    Parameters
    ----------
    pubkey : str
        hex or npub (bech32) pubkey

    Returns
    -------
    str
        hex pubkey
    """
    if pubkey.startswith('npub'):
         pubkey = \
            PublicKey.from_npub(pubkey).hex()
    return pubkey

In [ ]:
#| export

def get_notifyr_privkey() -> str:
    """returns a private key from keychain and
    sets a new one if one doesn't exist

    Returns
    -------
    str
        private key in hex format
    """
    privkey_hex = get_private_key()
    if privkey_hex is None:
        privkey_hex = PrivateKey().hex()
    set_private_key(privkey_hex)
    assert get_private_key() == privkey_hex
    return privkey_hex

In [ ]:
#| export

import functools

In [ ]:
#| export

def notifyr(func=None, recipient_pubkey: str = None, relay_urls: list[str] = None):
   """A decorator that will set a nostr private key to `func.notifyr_privkey_hex
   and use that key to send an encrypted message to it's own public key on the start
   and termination of the decorated function. The output will send whether the function
   runs completely or ends in an error with an informative message.

   Parameters
   ----------
   func : function
       the function to be decorated

   Returns
   -------
   function
       the decorated function

   Raises
   ------
   e
       if the function fails, else returns the function result
   """
   notifyr_privkey_hex = get_private_key()
   if notifyr_privkey_hex is None:
      notifyr_privkey_hex = PrivateKey().hex()
   set_private_key(notifyr_privkey_hex)
   assert get_private_key() == notifyr_privkey_hex
   if relay_urls is None:
      relay_urls = ['wss://relay.damus.io',
                    'wss://brb.io']
   if recipient_pubkey is None:
      recipient_pubkey_hex = \
         PrivateKey.from_hex(notifyr_privkey_hex).public_key.hex()
   else:
      if recipient_pubkey.startswith('npub'):
         recipient_pubkey_hex = \
            PublicKey.from_npub(recipient_pubkey).hex()
      else:
         recipient_pubkey_hex = recipient_pubkey

   if func is None:
        return lambda func: notifyr(func=func,
                                    recipient_pubkey=recipient_pubkey,
                                    relay_urls=relay_urls)

   @functools.wraps(func)
   def notifier(*args,**kwargs):
      notifyr_client = Client(private_key_hex=notifyr_privkey_hex,
                              relay_urls=relay_urls)
      notifyr_pubkey_hex = notifyr_client.public_key.hex()
      function_name = func.__name__
      message = f'**process name**: {function_name} started!'
      send_nostr_message(recipient_pubkey_hex=recipient_pubkey_hex,
                         notifyr_client=notifyr_client,
                         message=message)
      try:
         result = func(*args,**kwargs)
         message = f'**process name**: {function_name}\n' \
                   f'**finished** - preview of result:\n' \
                   f'-----------------------------\n\n'\
                   f'{str(result)[:100]}'
      except Exception as e:
         result = e
         message = f'**process name**: {function_name}\n' \
                   f'**failed** with error:\n\t{type(e).__name__}: {e}'
      send_nostr_message(recipient_pubkey_hex=recipient_pubkey_hex,
                         notifyr_client=notifyr_client,
                         message=message)
      if issubclass(type(result), Exception):
         raise result
      else:
         return result
   notifier.notifyr_private_key = notifyr_privkey_hex
   return notifier

In [ ]:
#| export

def notifyr(func=None, recipient_pubkey: str = None, relay_urls: list[str] = None):
   """A decorator that will set a nostr private key to `func.notifyr_privkey_hex
   and use that key to send an encrypted message to it's own public key on the start
   and termination of the decorated function. The output will send whether the function
   runs completely or ends in an error with an informative message.

   Parameters
   ----------
   func : function
       the function to be decorated

   Returns
   -------
   function
       the decorated function

   Raises
   ------
   e
       if the function fails, else returns the function result
   """
   notifyr_privkey_hex = get_notifyr_privkey()
   if relay_urls is None:
      relay_urls = ['wss://relay.damus.io',
                    'wss://brb.io']
   if recipient_pubkey is None:
      recipient_pubkey_hex = \
         PrivateKey.from_hex(notifyr_privkey_hex).public_key.hex()
   else:
      recipient_pubkey_hex = convert_to_hex(recipient_pubkey)
   notifyr_client = Client(private_key_hex=notifyr_privkey_hex,
                           relay_urls=relay_urls)
   if func is None:
        return lambda func: notifyr(func=func,
                                    recipient_pubkey=recipient_pubkey,
                                    relay_urls=relay_urls)
   @functools.wraps(func)
   def notifier(*args,**kwargs):
      function_name = func.__name__
      message = f'**process name**: {function_name} started!'
      send_nostr_message(recipient_pubkey_hex=recipient_pubkey_hex,
                         notifyr_client=notifyr_client,
                         message=message)
      try:
         result = func(*args,**kwargs)
         message = f'**process name**: {function_name}\n' \
                   f'**finished** - preview of result:\n' \
                   f'-----------------------------\n\n'\
                   f'{str(result)[:100]}'
      except Exception as e:
         result = e
         message = f'**process name**: {function_name}\n' \
                   f'**failed** with error:\n\t{type(e).__name__}: {e}'
      send_nostr_message(recipient_pubkey_hex=recipient_pubkey_hex,
                         notifyr_client=notifyr_client,
                         message=message)
      if issubclass(type(result), Exception):
         raise result
      else:
         return result
   notifier.notifyr_private_key = notifyr_privkey_hex
   return notifier

Now we can decorate a couple functions! 

In [ ]:
pubkey = PrivateKey.from_hex(get_private_key()).public_key.bech32()
@notifyr(recipient_pubkey=pubkey, relay_urls=['ws://127.0.0.1:6969'])
def success():
    return True

@notifyr
def another_success():
    return True

@notifyr
def raise_error():
    raise Exception('Oh no! Process failed!')


And test a successful function

In [ ]:
success()

/Users/ryanarmstrong/python/nostrfastr/nostrfastr/nostr.py:172: UserWarning: ws://127.0.0.1:6969 is not connected... removing relay.
  warnings.warn(
/Users/ryanarmstrong/python/nostrfastr/nostrfastr/client.py:479: UserWarning: the current implementation of messages should be used with caution
                    see https://github.com/nostr-protocol/nips/issues/107
  warnings.warn('''the current implementation of messages should be used with caution


True

In [ ]:
another_success()

/Users/ryanarmstrong/python/nostrfastr/nostrfastr/client.py:479: UserWarning: the current implementation of messages should be used with caution
                    see https://github.com/nostr-protocol/nips/issues/107
  warnings.warn('''the current implementation of messages should be used with caution


True

In [ ]:
another_success()

True

And test a failing function

In [ ]:
from fastcore.test import test_fail

In [ ]:
test_fail(raise_error)

/Users/ryanarmstrong/python/nostrfastr/nostrfastr/client.py:479: UserWarning: the current implementation of messages should be used with caution
                    see https://github.com/nostr-protocol/nips/issues/107
  warnings.warn('''the current implementation of messages should be used with caution


Remember we can retrieve the private key to log into our messages from the keychain or from `success.notifyr_privkey_hex` or `raise_error.notifyr_privkey_hex`.

In this case you will either have to trust me that it works... or verify for yourself!

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
success.notifyr_private_key

'b6a56d9a3aff4a5e248d9ed031d971bed4b73bd1764704c422413c776232a473'